# HW3: Assinatura Digital ElGamal

>### Grupo: 
>>André Araújo A87987
\
>>Miguel Gonçalves A90416
\
>>Paulo Costa A87986

## Descrição: 

>Pretende-se que implemente o esquema de assinatura digital RSA.
\
>Ver, por exemplo, pag 221 e seguintes de N Smart, "Cryptography: An Introduction 
(3rd Edition)" (ou ainda 9.1 de David R. Kohel, "Cryptography"); por forma a usar a secção 3, sugerimos que use a construção da síntese dada pela função de hash do sagemath. Por exemplo, a "hash" de uma mensagem pode ser obtida assim: \
>import hashlib \
>HH =hashlib.sha256(b"Nobody expects the Spanish Inquisition").hexdigest() \
>ZZ('0x'+HH)  \
\
>Neste caso, usámos a função de hash SHA3 com 256 bits; independentemente do tamanho da mensagem original, a função de hash tem como imagem uma mensagem com no máximo 256 bits. \
\
>Ver, também, https://en.wikipedia.org/wiki/RSA_(cryptosystem)#Signing_messages \
\
>Finalmente, explore a assinatura cega RSA, também denominada assinatura cega de Chaum: https://en.wikipedia.org/wiki/Blind_signature#Blind_RSA_signatures 
ou ainda 9.3 de David R. Kohel, "Cryptography"

## Introdução:
>>O algotirmo de criptografia RSA é particularmente interessante, pois pode ser usado diretamente como um algoritmo de assinatura com recuperação de mensagem . \
>>•O remetente aplica a decriptação RSA para gerar a assinatura, pegando na assinatura e elevada ao expoente privado d.    \
s =$ m^d$ (mod N). \
>>•O destinatário aplica a Encriptação RSA para recuperar a mensagem original.  \
m = $s^e$ (mod N).

In [117]:
def chave_RSA(nbits):#cria a chave com nbits bits
    p=random_prime(2^(nbits//2),2^(nbits//2-1))
    q=random_prime(2^(nbits//2+1),2^(nbits//2))
    n=p*q
    m=(p-1)*(q-1)
    Zm=IntegerModRing(m)
    Zn=IntegerModRing(n)
    e=Zm.random_element()
    while gcd(e,m)!=1:
        e=Zm.random_element()
    d=1/e
    return (n,e),(n,d)


In [118]:
PubKey,PrivKey=chave_RSA(32)
PubKey,PrivKey

((3784320389, 307035235), (3784320389, 378410779))

In [121]:
# Função que assina uma mensagem
def assina(m,PrivKey): 
    n,d=PrivKey
    Zn=IntegerModRing(n)
    s=Zn(m)^d
    return s

In [122]:
# Função que confirma a assinatura de uma mensagem
def verifica(s,PubKey): 
    n,e=PubKey
    Zn=IntegerModRing(n)
    m=Zn(s)^e
    return m

In [123]:
m=12345

In [124]:
s=assina(m,PrivKey)
s

3055372562

In [125]:
verifica(s,PubKey)

12345

# Esquema de assinatura RSA com Hash
>Suponhamos que recebemos uma mensagem muito longa m para assinar, primeiro calculamos h(m) e depois aplicamos a assinatura RSA a h(m), i.e. a assinatura é dada por 
$$s = h(m)^d (mod N)$$
>A assinatura e a mensagem são então enviadas como par (m,s). \
>Para verificar o par (m,s) usando uma função hash temos 3 etapas:
>>• Encriptar s usando a encriptação RSA para recuperar h"
, i.e.
   $h" = s^e (mod N).$ \
>>• Calcular h(m) de m.\
>>• Verificar se h" = h(m). Se verificar, aceitar a assinatura como válida, caso contrário a assinatura deve ser rejeitada.

In [160]:
# Função que assina uma mensagem atravez de uma função de hash
import hashlib
def assinahash(ms,PrivKey): 
    n,d=PrivKey
    HH =hashlib.sha256(ms.encode('utf-8')).hexdigest()
    m=ZZ('0x'+HH)
    Zn=IntegerModRing(n)
    s=Zn(m)^d
    return ms,s

In [161]:
# Função que confirma a assinatura de uma mensagem atravez de uma função de hash
def verificahash(p,PubKey): 
    n,e=PubKey
    ms,s=p
    Zn=IntegerModRing(n)
    m=Zn(s)^e
    HH =hashlib.sha256(ms.encode('utf-8')).hexdigest()
    m1=ZZ('0x'+HH)
    return m==m1

In [162]:
m="12345"
s=assinahash(m,PrivKey)
s

('12345', 2581399952)

In [163]:
verificahash(s,PubKey)

True

# Esquema de assinatura cega de Chaum
>O esquema de assinatura ceha de Chaum é um esquema baseado no RSA, adaptado para assinaturas cegas.
No protocolo a seguir, assumimos que o Bob configurou uma chave pública RSA (e,n) com a chave privada correspondente (d,n), de forma a que as funções de assinatura RSA do Bob são $S_B(m) = m^d$.
>>1. *Configuração inicial:* A Alice obtém a chave pública do Bob (e,n) e escolhe uma chave de sessão pública aleatória k, de modo a que 0 < k < n e gcd(k,n)=1.
>>2. *Blinding:* A Alice calcula $m^∗ = m.k^e$, e envia $m^∗$ para o Bob.
>>3. *Assinar:* O Bob calcula $s^∗ = m^{∗d}$ e envia de volta para a Alice.
>>4. *Unblinding:* A Alice calcula $s = k^{−1}s^∗$, que é equivalente a $S_B(m) = m^d$.

In [198]:
# Implemetação da 1ªparte da blind signature
import random
def blind1(m,PubKey): 
    n,e=PubKey
    k=n
    while gcd(k,n)!=1:
        k=random.randint(1, n)
    Zn=IntegerModRing(n)
    m1=m*Zn(k)^e
    return m1,k

In [199]:
# Implemetação da 2ªparte da blind signature
def blind2(m,PrivKey): 
    n,d=PrivKey
    s=Zn=IntegerModRing(n)
    s=Zn(m)^d
    return s

In [212]:
# Implemetação da 3ªparte da blind signature
def blind3(s,k,PubKey): 
    n,e=PubKey
    Zn=IntegerModRing(n)
    s=s*Zn(k)^-1
    return s

In [213]:
m1,k=blind1(123,PubKey)
m1

778116049

In [214]:
m2=blind2(m1,PrivKey)
m2

1813822902

In [217]:
s=blind3(m2,k,PubKey)
s

3751676701

In [219]:
s==assina(123,PrivKey) #verifica se esta correto

True

## Conclusão:

>Em estilo de conclusão, achamos que os objetivos do exercício foram alcançados. Isto, pois conseguimos implementar a assinatura digital pretendida.
\
\
>Esperemos que os professores também achem o mesmo e assim, damos como concluído o terceiro trabalho relativo à UC **Teoria de Números Computacional** e esperemos que continuemos com este sucesso no último trabalho!


